#<font color=blue>**YOLO를 통한 버클리 데이터셋 학습**</font>

##<font color=green>**버클리 데이터 업로드 진행**</font>

####<font color=red>구글 드라이브 업로드 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


####<font color=red>업로드한 BDD_data.zip 압축풀기

In [ ]:
!unzip /content/drive/MyDrive/DataSet.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: DataSet/labels/train/0402.txt  
  inflating: DataSet/labels/train/0403.txt  
  inflating: DataSet/labels/train/0404.txt  
  inflating: DataSet/labels/train/0405.txt  
  inflating: DataSet/labels/train/0406.txt  
  inflating: DataSet/labels/train/0407.txt  
  inflating: DataSet/labels/train/0408.txt  
  inflating: DataSet/labels/train/0409.txt  
  inflating: DataSet/labels/train/0410.txt  
  inflating: DataSet/labels/train/0411.txt  
  inflating: DataSet/labels/train/0412.txt  
  inflating: DataSet/labels/train/0413.txt  
  inflating: DataSet/labels/train/0414.txt  
  inflating: DataSet/labels/train/0415.txt  
  inflating: DataSet/labels/train/0416.txt  
  inflating: DataSet/labels/train/0417.txt  
  inflating: DataSet/labels/train/0418.txt  
  inflating: DataSet/labels/train/0419.txt  
  inflating: DataSet/labels/train/0420.txt  
  inflating: DataSet/labels/train/0421.txt  
  inflating: DataSet/labels/train/0422.txt  
  inflating: DataSe

####<font color=red>각각의 데이터 개수 확인

In [ ]:
import os

images_train=os.listdir('/content/DataSet/images/train/')
images_val=os.listdir('/content/DataSet/images/val/')

labels_train=os.listdir('/content/DataSet/labels/train/')
labels_val=os.listdir('/content/DataSet/labels/val/')


print(f'train images 개수:{len(images_train)}')
print(f'validation images 개수:{len(images_val)}')
print("")
print(f'train labels 개수:{len(labels_train)}')
print(f'validation labels 개수:{len(labels_val)}')

##<font color=green>**git의 YOLOv5 설치**</font>
설치 후 custom_data.yaml 파일을 yolov5/data 폴더에 업로드하기

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (CPU)


##<font color=green>**학습 시작**</font>


In [ ]:
# Train YOLOv5s on custom_data for 20 epochs
!python train.py --img 640 --batch 16 --epochs 10 --data custom_data.yaml --weights yolov5x.pt --cache

train: weights=yolov5x.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, 

In [ ]:
!ls runs/train/exp

####<font color=red>학습된 Loss 그래프 확인

In [ ]:
from IPython.display import Image as Display # to display images
Display(filename='runs/train/exp/results.png', width=1080)

####<font color=red>텐서보드를 통해 Loss 그래프 확인

In [ ]:
%reload_ext tensorboard
# TensorBoard 준비하기
%load_ext tensorboard
%tensorboard --logdir ./runs

####<font color=red>원하는 이미지로 테스트</font>

In [ ]:
# Evaluation with the validation data
# 원하는 이미지의 경로와 파일명을 지정하여 실행
!python detect.py --weights best.pt --img 640 --conf 0.25 --source ../DataSet/images/val/4201.jpg

##<font color=green>**Weight 검증**</font>

In [ ]:
!python val.py --weights test/s_best.pt --data custom_data.yaml --img 640 --iou 0.6 --half

val: data=./data/custom_data.yaml, weights=['test/s_best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 224 layers, 7078183 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning '../DataSet/labels/val' images and labels...1200 found, 0 missing, 0 empty, 0 corrupted: 100% 1200/1200 [00:01<00:00, 700.62it/s]
val: New cache created: ../DataSet/labels/val.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 38/38 [05:57<00:00,  9.40s/it]
                 all       1200      21736      0.634      0.329      0.343      0.155
        traffic sign       1200       3327      0.536      0.402      0.381      0.111
       traffic light       1200       3988      0.609

##<font color=green>**대회 결과 다운로드**</font>
Loss 그래프

yaml 파일

학습중 베스트 모델

<<경로 주의>>

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/train/exp/results.png')
files.download('/content/yolov5/data/custom_data.yaml')
files.download('/content/yolov5/runs/train/exp/weights/best.pt')
